In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import streamlit as st

In [135]:
df = pd.read_csv("icici tick/100.csv")
symbol = df.values

In [138]:
symbol[0][0]

'ADANIENT'

In [10]:
import warnings
from datetime import datetime, timedelta
import pytz
from breeze_connect import BreezeConnect
from snapi_py_client.snapi_bridge import StocknoteAPIPythonBridge
import pandas as pd
import json
import time
import threading
import pytz
import warnings
import numpy as np
# Turn off pandas warnings
warnings.filterwarnings("ignore")


api_key = "62M23)75L223000x45Kb7TG3O0261368"
sec_key = "4646x6992821&p7Cy1+2z87g93u89935"
api_session = '8083172' #ICICI -URL: https://api.icicidirect.com/apiuser/login?api_key=62M23%2975L223000x45Kb7TG3O0261368

# Initialize SDK
breeze = BreezeConnect(api_key=api_key)
breeze.generate_session(api_secret=sec_key,
                        session_token=api_session)



In [124]:
breeze.get_names(exchange_code = 'NSE',stock_code = 'MCDOWELL-N')

{'exchange_code': 'NSE',
 'exchange_stock_code': 'MCDOWELL-N',
 'isec_stock_code': 'UNISPI',
 'isec_token': '10447',
 'company name': 'UNITED SPIRITS LIMITED',
 'isec_token_level1': '4.1!10447',
 'isec_token_level2': '4.2!10447'}

In [94]:
from datetime import datetime, timedelta
from sklearn.neighbors import KNeighborsRegressor
import ta

today = datetime.today() # Get today's date
one_month_ago = today - timedelta(days=365) # Subtract 30 days to get date 1 month ago
from_date = one_month_ago.strftime('%Y-%m-%dT%H:%M:%S.000Z') # Format the date as a string

to_date = today.strftime('%Y-%m-%dT%H:%M:%S.000Z') 
breeze.get_names(exchange_code = 'NSE',stock_code = 'TATASTEEL')
data = breeze.get_historical_data_v2(interval="1day",
                            from_date= from_date,
                            to_date= to_date,
                            stock_code="fgh",
                            exchange_code="NSE",
                            product_type="cash")


In [96]:
print(data['Error'])

{'Error': None, 'Status': 200, 'Success': []}

In [ ]:

data = pd.DataFrame(data['Success'])
data[['open', 'high', 'low', 'close', 'volume']] = data[['open', 'high', 'low', 'close', 'volume']].astype(float)
data = data.set_index('datetime')
data

data['Close_next'] = data['close'].shift(-1)
data['Close_pct_change'] = data['Close_next'].pct_change()
data = data.dropna()
 # Calculate technical indicators
data = pd.DataFrame(ta.add_all_ta_features(data, open="open", high="high", low="low", close="close", volume="volume", fillna=True))
index = pd.DataFrame(data.columns)
index = index[0].to_numpy()[8:]
index = index.tolist()
X = data[index].values
y = data[['Close_next', 'Close_pct_change']].values

# Train the KNN model
model = KNeighborsRegressor()
model.fit(X, y)

# Prepare feature data for the next day
last_day_data = data.iloc[-1][index].values.reshape(1, -1)  # Use the last day's technical indicator values

# Predict the next closing price and percentage change
next_close, next_pct_change = model.predict(last_day_data)[0]
print(f'close next: {next_close}')
print(f'close %: {next_pct_change*100}%')

In [141]:
import ta
import yfinance as yf
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd
import requests
from datetime import date, timedelta
from time import sleep

class Prediction:
    
    @staticmethod
    def send_message(bot_token, bot_chatID, message):
        url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
        data = {"chat_id": bot_chatID, "text": message, "parse_mode": "Markdown"}
        response = requests.post(url, data=data)
        return response.json()

    @staticmethod
    def predict_next_close(sym):

        today = datetime.today() # Get today's date
        one_month_ago = today - timedelta(days=365) # Subtract 30 days to get date 1 month ago
        from_date = one_month_ago.strftime('%Y-%m-%dT%H:%M:%S.000Z') # Format the date as a string

        to_date = today.strftime('%Y-%m-%dT%H:%M:%S.000Z') 
        breeze.get_names(exchange_code = 'NSE',stock_code = 'TATASTEEL')
        data = breeze.get_historical_data_v2(interval="1day",
                                    from_date= from_date,
                                    to_date= to_date,
                                    stock_code=sym,
                                    exchange_code="NSE",
                                    product_type="cash")
        print(data['Success'])
        data = pd.DataFrame(data['Success'])
        data[['open', 'high', 'low', 'close', 'volume']] = data[['open', 'high', 'low', 'close', 'volume']].astype(float)
        data = data.set_index('datetime')

        data['Close_next'] = data['close'].shift(-1)
        data['Close_pct_change'] = data['Close_next'].pct_change()
        data = data.dropna()
        # Calculate technical indicators
        data = pd.DataFrame(ta.add_all_ta_features(data, open="open", high="high", low="low", close="close", volume="volume", fillna=True))
        index = pd.DataFrame(data.columns)
        index = index[0].to_numpy()[8:]
        index = index.tolist()
        X = data[index].values
        y = data[['Close_next', 'Close_pct_change']].values

        # Train the KNN model
        model = KNeighborsRegressor()
        model.fit(X, y)

        # Prepare feature data for the next day
        last_day_data = data.iloc[-1][index].values.reshape(1, -1)  # Use the last day's technical indicator values

        # Predict the next closing price and percentage change
        next_close, next_pct_change = model.predict(last_day_data)[0]
        sleep(3)
        return next_close, next_pct_change


if __name__ == '__main__':
    
    bot_token = '6111156932:AAGWg7uRvD7cwTK_UpzGcWv54MJkKWgyMoI'
    bot_chatID = '5562607566'
    today = date.today()
    two_days_later = today + timedelta(days=1)
    message = f"predictions for {two_days_later} using Day timeframe"
    Prediction.send_message(bot_token, bot_chatID, message)
    df = pd.read_csv("icici tick/100.csv")
    symbol = df.values
    for i in range(94):
        sym = symbol[i][0]
        prediction = Prediction.predict_next_close(sym)
        if prediction and abs(prediction[1]) > 1.0:
            message = f"{sym}\nchange: {prediction[1]:.2f}%"
            Prediction.send_message(bot_token, bot_chatID, message)


[]


KeyError: "None of [Index(['open', 'high', 'low', 'close', 'volume'], dtype='object')] are in the [columns]"